In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # Visualization
import seaborn as sns #data visualization
import time
import sklearn.metrics as m #predictive data analysis 

In [2]:
conda install -c glemaitre imbalanced-learn

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed

Note: you may need to restart the kernel to use updated packages.



Examining imbalanced-learn:  80%|########  | 4/5 [00:00<00:00, 67.67it/s]
                                                                         

Examining conflict for python imbalanced-learn:   0%|          | 0/5 [00:00<?, ?it/s]
                                                                                     

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - imbalanced-learn -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.9

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indicates that the thing on the left is somehow
not available for the python version you are constrained to. Note that conda will not
change your python version to a different minor version unless you explicitly specify
that.





In [3]:
!pip install imblearn

In [4]:
from imblearn.over_sampling import SMOTE
import imblearn
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Load Data
my_dataset=pd.read_csv("my_dataset.csv")
my_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600100 entries, 0 to 600099
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   IPV4_SRC_ADDR               600100 non-null  object 
 1   L4_SRC_PORT                 600100 non-null  int64  
 2   IPV4_DST_ADDR               600100 non-null  object 
 3   L4_DST_PORT                 600100 non-null  int64  
 4   PROTOCOL                    600100 non-null  int64  
 5   L7_PROTO                    600100 non-null  float64
 6   IN_BYTES                    600100 non-null  int64  
 7   OUT_BYTES                   600100 non-null  int64  
 8   IN_PKTS                     600100 non-null  int64  
 9   OUT_PKTS                    600100 non-null  int64  
 10  TCP_FLAGS                   600100 non-null  int64  
 11  FLOW_DURATION_MILLISECONDS  600100 non-null  int64  
 12  Label                       600100 non-null  int64  
 13  Attack        

In [6]:
#Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
my_dataset.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.100.6,52670,192.168.100.1,53,17,5.212,71,126,1,1,0,4294966,0,Benign
1,192.168.100.6,49160,192.168.100.149,4444,6,0.000,217753000,199100,4521,4049,24,4176249,1,Theft
2,192.168.100.46,3456,192.168.100.5,80,17,0.000,8508021,8918372,9086,9086,0,4175916,0,Benign
3,192.168.100.3,80,192.168.100.55,8080,6,7.000,8442138,9013406,9086,9086,0,4175916,0,Benign
4,192.168.100.46,80,192.168.100.5,80,6,7.000,8374706,0,9086,0,0,4175916,0,Benign


In [7]:
drop_cols = []
for i in my_dataset.columns:
    if len(my_dataset[i].unique())==1:
        drop_cols.append(i)
print("Total columns with only 1 unique value:", len(drop_cols))
my_dataset.drop(drop_cols, 1, inplace=True)

Total columns with only 1 unique value: 0


In [8]:
my_dataset.dropna(1,inplace=True)

In [9]:
#Split dataset on train and test
from sklearn.model_selection import train_test_split
train, test=train_test_split(my_dataset,test_size=0.3, random_state=10)

In [10]:
#Split dataset on train and test
from sklearn.model_selection import train_test_split
train, test=train_test_split(my_dataset,test_size=0.3, random_state=10)

#Exploratory Analysis
# Descriptive statistics
train.describe()
test.describe()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label
count,180030.000000,180030.000000,180030.00000,180030.000000,1.800300e+05,1.800300e+05,180030.000000,180030.000000,180030.000000,1.800300e+05,180030.000000
mean,46550.677265,7975.248192,6.58182,8.989024,8.570278e+03,4.940851e+03,12.543276,5.736005,21.865861,3.475628e+06,0.976693
std,12016.939538,14132.240182,2.55893,35.045190,2.221175e+05,3.913169e+05,245.665646,169.866978,8.108470,1.660000e+06,0.150878
min,0.000000,0.000000,1.00000,0.000000,2.800000e+01,0.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,39210.000000,80.000000,6.00000,0.000000,4.400000e+01,4.000000e+01,1.000000,1.000000,22.000000,4.278545e+06,1.000000
50%,47892.000000,1875.000000,6.00000,0.000000,4.400000e+01,4.000000e+01,1.000000,1.000000,22.000000,4.294966e+06,1.000000
75%,55307.000000,8009.000000,6.00000,7.000000,1.120000e+02,4.000000e+01,2.000000,1.000000,22.000000,4.294967e+06,1.000000
max,65535.000000,65535.000000,17.00000,244.000000,9.519928e+06,1.513069e+08,9598.000000,13807.000000,214.000000,4.294967e+06,1.000000


In [11]:
# Packet Attack Distribution
train['Label'].value_counts()
test['Label'].value_counts()

1    175834
0      4196
Name: Label, dtype: int64

In [12]:
#Scalling numerical attributes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# extract numerical attributes and scale it to have zero mean and unit variance  
cols = train.select_dtypes(include=['float64','int64']).columns
sc_train = scaler.fit_transform(train.select_dtypes(include=['float64','int64']))
sc_test = scaler.fit_transform(test.select_dtypes(include=['float64','int64']))
# turn the result back to a dataframe
sc_traindf = pd.DataFrame(sc_train, columns = cols)
sc_testdf = pd.DataFrame(sc_test, columns = cols)


In [14]:
# importing one hot encoder from sklearn 
from sklearn.preprocessing import OneHotEncoder 
# creating one hot encoder object 
onehotencoder = OneHotEncoder() 
trainDep = train['Label'].values.reshape(-1,1)
trainDep = onehotencoder.fit_transform(trainDep).toarray()
testDep = test['Label'].values.reshape(-1,1)
testDep = onehotencoder.fit_transform(testDep).toarray()

print (testDep)
train_X=sc_traindf
train_y=trainDep[:,0]

test_X=sc_testdf
test_y=testDep[:,0]

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [15]:
#Feature Selection
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier();

# fit random forest classifier on the training set
rfc.fit(train_X, train_y);

# extract important features
score = np.round(rfc.feature_importances_,3)
importances = pd.DataFrame({'feature':train_X.columns,'importance':score})
importances = importances.sort_values('importance',ascending=False).set_index('feature')


In [16]:
#Recursive feature elimination
from sklearn.feature_selection import RFE
import itertools

rfc = RandomForestClassifier()

# create the RFE model and select 10 attributes
rfe = RFE(rfc, n_features_to_select=11)
rfe = rfe.fit(train_X, train_y)

# summarize the selection of the attributes
feature_map = [(i, v) for i, v in itertools.zip_longest(rfe.get_support(), train_X.columns)]
selected_features = [v for i, v in feature_map if i==True]

selected_features

a = [i[0] for i in feature_map]
train_X = train_X.iloc[:,a]
test_X = test_X.iloc[:,a]

#Dataset Partition
X_train,X_test,Y_train,Y_test = train_test_split(train_X,train_y,train_size=0.70, random_state=2)

In [17]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: google-auth-oauthlib<0.5,>=0.4.1 in c:\users\bukola\anaconda3\lib\site-packages (from tensorboard~=2.6->tensorflow) (0.4.6)



In [18]:
#Fitting Models
#importing keras libraries and packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [19]:
#Fitting Models
# Importing keras libraries and packahes
import keras
from keras.models import Sequential
from keras.layers import Dense

In [20]:
#initializing the ANN
classifier = Sequential()

In [21]:
#Adding the input layer and the first hidden layer
classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

In [22]:
#Adding the second hidden layer
classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'relu'))

In [23]:
#Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [24]:
#Compiling the ANN
classifier.compile(optimizer ='adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
#Fitting thE ANN to the training set
classifier.fit(X_train, Y_train, batch_size = 10, epochs = 30)

Epoch 1/30
29405/29405 [==============================] - 17s 557us/step - loss: 0.0045 - accuracy: 0.9997
Epoch 2/30
29405/29405 [==============================] - 16s 556us/step - loss: 6.7709e-05 - accuracy: 1.0000
Epoch 3/30
29405/29405 [==============================] - 16s 550us/step - loss: 8.8821e-06 - accuracy: 1.0000
Epoch 4/30
29405/29405 [==============================] - 16s 555us/step - loss: 2.0632e-08 - accuracy: 1.0000
Epoch 5/30
29405/29405 [==============================] - 16s 553us/step - loss: 1.2667e-09 - accuracy: 1.0000
Epoch 6/30
29405/29405 [==============================] - 16s 544us/step - loss: 2.2182e-10 - accuracy: 1.0000
Epoch 7/30
29405/29405 [==============================] - 16s 552us/step - loss: 1.1284e-10 - accuracy: 1.0000
Epoch 8/30
29405/29405 [==============================] - 16s 558us/step - loss: 8.3433e-11 - accuracy: 1.0000
Epoch 9/30
29405/29405 [==============================] - 16s 553us/step - loss: 7.1248e-11 - accuracy: 1.0000
Epoch

In [26]:
K_train = (classifier.predict(X_train) > 0.5)
K_test = (classifier.predict(X_test) > 0.5)

In [27]:

#Predicting for test data
pred_ann = classifier.predict(X_test)

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [29]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer ='adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier


In [30]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 30)
scores = cross_val_score(estimator = classifier, X = X_train, y= Y_train, cv = 10, n_jobs = 1)

Epoch 1/30
26465/26465 [==============================] - 16s 583us/step - loss: 0.0061 - accuracy: 0.9997
Epoch 2/30
26465/26465 [==============================] - 16s 589us/step - loss: 6.5253e-05 - accuracy: 1.0000
Epoch 3/30
26465/26465 [==============================] - 16s 587us/step - loss: 2.1460e-06 - accuracy: 1.0000
Epoch 4/30
26465/26465 [==============================] - 15s 570us/step - loss: 2.1276e-06 - accuracy: 1.0000
Epoch 5/30
26465/26465 [==============================] - 26s 1ms/step - loss: 7.1326e-05 - accuracy: 1.0000
Epoch 6/30
26465/26465 [==============================] - 44s 2ms/step - loss: 5.6505e-05 - accuracy: 1.0000
Epoch 7/30
26465/26465 [==============================] - 40s 2ms/step - loss: 1.6223e-06 - accuracy: 1.0000
Epoch 8/30
26465/26465 [==============================] - 47s 2ms/step - loss: 1.3526e-07 - accuracy: 1.0000
Epoch 9/30
26465/26465 [==============================] - 48s 2ms/step - loss: 7.4839e-09 - accuracy: 1.0000
Epoch 10/30
264

In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [32]:
X_train,X_test,Y_train,Y_test = train_test_split(train_X,train_y,train_size=0.70, random_state=2)


In [33]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, Y_train)
clf.predict_proba(X_test[:1])

array([[9.99999899e-01, 1.00849528e-07]])

In [34]:
clf.predict(X_test[:11])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [35]:
clf.score(X_test, Y_test)

1.0

In [41]:
#Evaluate Models
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import cross_val_score

models = []
models.append(('ANN', clf))


for i, v in models:
    start_time = time.time()
    scores = cross_val_score(v, X_train, Y_train, cv=10)
    accuracy = metrics.accuracy_score(Y_train, v.predict(X_train))
    confusion_matrix = metrics.confusion_matrix(Y_train, v.predict(X_train))
    classification = metrics.classification_report(Y_train, v.predict(X_train))
    MAE = metrics.mean_absolute_error(Y_train, v.predict(X_train))
    R2_Score = metrics.r2_score(Y_train, v.predict(X_train))
    F1_Score = metrics.f1_score(Y_train, v.predict(X_train))
    end_time = time.time()
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()
    print("Mean Absolute Error:" "\n", MAE) 
    print()
    print("R2_score:" "\n", R2_Score ) 
    print()
    print("F1_score:" "\n", F1_Score ) 
    print()
    print("Testing time: ",end_time-start_time)
    print()
    


============================== ANN Model Evaluation ==============================

Cross Validation Mean Score:
 0.99999659921782

Model Accuracy:
 1.0

Confusion matrix:
 [[287283      0]
 [     0   6766]]

Classification report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    287283
         1.0       1.00      1.00      1.00      6766

    accuracy                           1.00    294049
   macro avg       1.00      1.00      1.00    294049
weighted avg       1.00      1.00      1.00    294049


Mean Absolute Error:
 0.0

R2_score:
 1.0

F1_score:
 1.0

Testing time:  114.9535596370697



In [42]:
#Validate Models

for i, v in models:
    start_time = time.time()
    accuracy = metrics.accuracy_score(Y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(Y_test, v.predict(X_test))
    classification = metrics.classification_report(Y_test, v.predict(X_test))
    MAE = metrics.mean_absolute_error(Y_test, v.predict(X_test))
    R2_Score = metrics.r2_score(Y_test, v.predict(X_test))
    F1_Score = metrics.f1_score(Y_test, v.predict(X_test))
    end_time = time.time()
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()
    print("Mean Absolute Error:" "\n", MAE)  
    print()
    print("R2_score:" "\n", R2_Score ) 
    print()
    print("F1_score:" "\n", F1_Score ) 
    print()
    print("Testing time: ",end_time-start_time)
    print()


============================== ANN Model Test Results ==============================

Model Accuracy:
 1.0

Confusion matrix:
 [[123124      0]
 [     0   2897]]

Classification report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    123124
         1.0       1.00      1.00      1.00      2897

    accuracy                           1.00    126021
   macro avg       1.00      1.00      1.00    126021
weighted avg       1.00      1.00      1.00    126021


Mean Absolute Error:
 0.0

R2_score:
 1.0

F1_score:
 1.0

Testing time:  1.2198066711425781

